In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
crime = pd.read_csv('sample_Chicago_Crimes.csv')

# 删除存在nan值的行
crime.dropna(inplace=True)

# 删除2017年数据
crime = crime[crime['Year'] != 2017]

# 删除无效列
delete_list = ['Unnamed: 0', 'ID', 'Case Number', 'IUCR', 'Location', 'X Coordinate', 'Y Coordinate', 'Updated On']
crime_analyse = crime.drop(columns=delete_list)

# 将时间数据转化成datatime
crime_analyse.Date = pd.to_datetime(crime_analyse.Date, format='%m/%d/%Y %I:%M:%S %p')

In [70]:
crime_analyse.sort_values(by='Date', inplace=True)

,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude
77488,2012-01-01 03:00:00,014XX W CULLERTON ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,1222,12.0,25.0,31.0,08B,2012,41.855144,-87.661808
77931,2012-01-01 19:19:00,014XX N CLARK ST,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,STREET,True,False,1821,18.0,42.0,8.0,08B,2012,41.908059,-87.631574
78100,2012-01-01 20:00:00,010XX N WESTERN AVE,THEFT,$500 AND UNDER,STREET,False,False,1312,12.0,32.0,24.0,06,2012,41.899959,-87.686952
78147,2012-01-01 21:30:00,069XX S INDIANA AVE,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,322,3.0,6.0,69.0,07,2012,41.768106,-87.620180
79954,2012-01-02 06:10:00,076XX S PULASKI RD,CRIMINAL DAMAGE,TO PROPERTY,OTHER,False,False,833,8.0,13.0,65.0,14,2012,41.754879,-87.722171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442947,2016-12-30 16:00:00,008XX N RICHMOND ST,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1211,12.0,26.0,24.0,26,2016,41.896972,-87.700407
1442656,2016-12-30 16:31:00,042XX W SCHOOL ST,ASSAULT,SIMPLE,STREET,True,False,1731,17.0,30.0,16.0,08A,2016,41.940842,-87.732719
1443378,2016-12-31 13:30:00,034XX S HALSTED ST,THEFT,$500 AND UNDER,LIBRARY,False,False,913,9.0,11.0,60.0,06,2016,41.832754,-87.646154
1443427,2016-12-31 17:50:00,036XX W 83RD PL,ROBBERY,ARMED: HANDGUN,SMALL RETAIL STORE,False,False,834,8.0,18.0,70.0,03,2016,41.741397,-87.713619
